In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models
from torch.utils.data import TensorDataset, DataLoader
import torch.nn.functional as F
import numpy as np
from sklearn.preprocessing import scale
import json

# Loading dataset

In [ ]:
dataset = np.loadtxt("../build neural networks/diabetes.csv", delimiter=",", skiprows=1 )
diabetes_X = dataset[:,0:8]
diabetes_Y = np.array(dataset[:,8]).astype(np.float32)
diabetes_X = np.array(scale(diabetes_X,axis=0)).astype(np.float32)

# Building the network

In [ ]:
input_size = 8
hidden_size = 8
output_size = 1

# Build a feed-forward network
model = nn.Sequential(nn.Conv2d(16, 33, (3, 5), stride=(2, 1), padding=(4, 2), dilation=(3, 1)),
                      nn.AvgPool2d((3, 2), stride=(2, 1)),
                      nn.Linear(input_size, hidden_size, bias=False),
                      nn.ReLU(),
                      nn.Linear(hidden_size, output_size),
                      nn.Softmax())

# Train the network

In [ ]:
inputs = torch.from_numpy(diabetes_X)
targets = torch.from_numpy(diabetes_Y)

train_ds = TensorDataset(inputs, targets)

In [ ]:
batch_size = 10
train_dl = DataLoader(train_ds, batch_size, shuffle=True)

In [ ]:
opt = torch.optim.SGD(model.parameters(), lr=1e-5)
loss_fn = F.mse_loss
loss = loss_fn(model(inputs.float()), targets)

In [ ]:
def fit(num_epochs, model, loss_fn, opt):
    for epoch in range(num_epochs):
        for xb,yb in train_dl:
            # Generate predictions
            pred = model(xb.float())
            loss = loss_fn(pred, yb)
            # Perform gradient descent
            loss.backward()
            opt.step()
            opt.zero_grad()
    print('Training loss: ', loss_fn(model(inputs), targets))

In [ ]:
fit(100, model, loss_fn, opt)

In [ ]:
model.state_dict()

# Save the network

In [ ]:
torch.save(model, 'pytorch_2_layer_diabetes.pt')

# Load network

In [ ]:
z = torch.load('pytorch_2_layer_diabetes.pt')

# Read network structure and information

In [ ]:
for layer in z:
    print(type(layer)) # type of layer, Activation functions are own layers here
    #print(layer.__dict__)
    if (type(layer)==torch.nn.modules.linear.Linear):
        print(layer.in_features) # input_size
        print(layer.out_features) # output_size --> equals units
        #print(layer.weights) # weights
        if (layer.bias is not None):
            print(layer.bias.detach().numpy()) # bias is None if use_bias=False else bias is not None
    elif (type(layer)==torch.nn.modules.conv.Conv2d):
        print(layer.in_channels) # input_size
        print(layer.out_channels) # output_size
        print(layer.kernel_size) # kernel_size
        print(layer.stride) # stride_size
        print(layer.padding) # padding_size
        print(layer.dilation) # dilation_Size
        print(layer.bias.detach().numpy()) # bias is None if use_bias=False else bias is not None
    elif (type(layer)==torch.nn.modules.pooling.MaxPool2d):
        print(layer.kernel_size) # pool_size
        print(layer.stride) # stride_size
        print(layer.padding) # padding_size
        print(layer.dilation) # dilation_Size
    print('-------')

In [ ]:
z.state_dict() # weights and biases

In [ ]:
print(z)

In [ ]:
counter=0
output = { "class_name":"Sequential", "config":{"name":"sequential_1", "layers":[]}}
for layer in model:
    out_layer = dict()
    out_layer["class_name"] = str(type(layer).__name__)
    if (type(layer)==torch.nn.modules.activation.ReLU or type(layer)==torch.nn.modules.activation.Sigmoid
        or type(layer)==torch.nn.modules.activation.Tanh or type(layer)==torch.nn.modules.activation.Softmax):
        output["config"]["layers"][counter-1]["activation"] = str(type(layer).__name__).lower()
        out_layer = None
        counter = counter -1
    elif (type(layer)==torch.nn.modules.linear.Linear):
        if (counter==0):
            out_layer["batch_input_shape"] = ['null', layer.in_features]
        out_layer["units"]= layer.out_features
        out_layer["kernel_values"] = layer.weight.detach().numpy().tolist()
        if (layer.bias is not None):
            out_layer["bias_values"] = layer.bias.detach().numpy().tolist()
        out_layer["activation"]="linear"
    elif (type(layer)==torch.nn.modules.conv.Conv2d or type(layer)==torch.nn.modules.conv.Conv1d):
        if (counter==0):
            out_layer["batch_input_shape"] = layer.in_channels
        out_layer["kernel_size"] = list(tuple(layer.kernel_size))
        out_layer["strides"] = list(tuple(layer.stride))
        out_layer["padding"] = "valid" if layer.padding==0 else "same"
        out_layer["dilation"] = list(tuple(layer.dilation))
    elif (type(layer)==torch.nn.modules.pooling.MaxPool2d or type(layer)==torch.nn.modules.pooling.MaxPool1d
        or type(layer)==torch.nn.modules.pooling.AvgPool2d or type(layer)==torch.nn.modules.pooling.AvgPool1d):
        if (counter==0):
            out_layer["batch_input_shape"] = layer.in_channels
        out_layer["kernel_size"] = list(tuple(layer.kernel_size))
        out_layer["strides"] = list(tuple(layer.stride))
        out_layer["padding"] = "valid" if layer.padding==0 else "same"
        if (type(layer)==torch.nn.modules.pooling.MaxPool2d or type(layer)==torch.nn.modules.pooling.MaxPool1d):
            out_layer["dilation"] = [layer.dilation]

    if (out_layer is not None):
        output["config"]["layers"].append(out_layer)
    counter=counter+1

print(json.dumps(output))